# Fairness analyses

Kendra Wyant  
June 11, 2025

### Set Up Environment

In [ ]:
#| message: false
#| warning: false

# handle conflicts
options(conflicts.policy = "depends.ok")
devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/fun_ml.R?raw=true")

ℹ SHA-1 hash of file is "32a0bc8ced92c79756b56ddcdc9a06e639795da6"

In [ ]:
#| message: false
#| warning: false

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(tidymodels))
suppressPackageStartupMessages(library(tidyposterior))
library(kableExtra, exclude = "group_rows")
library(Rcpp, exclude = "populate")
library(brms, exclude = c("ar", "mixture"))

Loading 'brms' package (version 2.22.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').

In [ ]:
#| output: false

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")

ℹ SHA-1 hash of file is "de12d764438078a9341db9bc0b2472c87e0ae846"

ℹ SHA-1 hash of file is "7be28854408fb58990ee996339c3258c133eed75"

In [ ]:
path_processed <- format_path(str_c("risk/data_processed/lag"))
path_models_lag <- format_path(str_c("risk/models/lag"))

### Read in Model Performance Metrics

In [ ]:
auroc_dem_0 <- read_csv(here::here(path_models_lag, 
                                   "test_auroc_6_x_5_1day_0_v3_nested_strat_lh_fairness.csv"),
                      col_types = cols()) |> 
  mutate(fold_num = rep(1:5, 6),
         repeat_num = c(rep(1, 5), rep(2, 5), rep(3, 5), 
                        rep(4, 5), rep(5, 5), rep(6, 5)),
         `not white` = if_else(`not white` < .001, .001, `not white`)) |>
  select(-outer_split_num)

auroc_dem_24 <- read_csv(here::here(path_models_lag, 
                                   "test_auroc_6_x_5_1day_24_v3_nested_strat_lh_fairness.csv"),
                      col_types = cols()) |> 
  mutate(fold_num = rep(1:5, 6),
         repeat_num = c(rep(1, 5), rep(2, 5), rep(3, 5), 
                        rep(4, 5), rep(5, 5), rep(6, 5)),
         `not white` = if_else(`not white` < .001, .001, `not white`)) |>
  select(-outer_split_num)

auroc_dem_72 <- read_csv(here::here(path_models_lag, 
                                   "test_auroc_6_x_5_1day_72_v3_nested_strat_lh_fairness.csv"),
                      col_types = cols()) |> 
  mutate(fold_num = rep(1:5, 6),
         repeat_num = c(rep(1, 5), rep(2, 5), rep(3, 5), 
                        rep(4, 5), rep(5, 5), rep(6, 5)),
         `not white` = if_else(`not white` < .001, .001, `not white`)) |>
  select(-outer_split_num)

auroc_dem_168 <- read_csv(here::here(path_models_lag, 
                                   "test_auroc_6_x_5_1day_168_v3_nested_strat_lh_fairness.csv"),
                      col_types = cols()) |> 
  mutate(fold_num = rep(1:5, 6),
         repeat_num = c(rep(1, 5), rep(2, 5), rep(3, 5), 
                        rep(4, 5), rep(5, 5), rep(6, 5)),
         `not white` = if_else(`not white` < .001, .001, `not white`)) |>
  select(-outer_split_num)

auroc_dem_336 <- read_csv(here::here(path_models_lag, 
                                   "test_auroc_6_x_5_1day_336_v3_nested_strat_lh_fairness.csv"),
                      col_types = cols()) |> 
  mutate(fold_num = rep(1:5, 6),
         repeat_num = c(rep(1, 5), rep(2, 5), rep(3, 5), 
                        rep(4, 5), rep(5, 5), rep(6, 5)),
         `not white` = if_else(`not white` < .001, .001, `not white`)) |>
  select(-outer_split_num)

### Get Median Posterior Probabilities and contrast analyses

function

In [ ]:
calc_pp <- function (lag, dem_var) {
  data_name <- str_c("auroc_dem_", lag)
  
  data <- 
    if (dem_var == "sex") {
    get(data_name) |> 
    select(id = repeat_num, id2 = fold_num, male, female)
  } else if (dem_var == "income") {
    get(data_name) |> 
    select(id = repeat_num, id2 = fold_num, `above poverty`, `below poverty`)
  } else if (dem_var == "race") {
    get(data_name) |> 
    select(id = repeat_num, id2 = fold_num, `non-hispanic white` = white, `not white`)
  } else {
    stop(dem_var, " not in data")
  }
  
  
  set.seed(101)
  pp <- data |> 
    perf_mod(formula = statistic ~ model + (1 | id2/id),
             transform = tidyposterior::logit_trans,  
             iter = 4000, chains = 4,  
             adapt_delta = .999,
             family = gaussian) 

  pp_tidy <- pp |> 
    tidy(seed = 123) |> 
    mutate(lag = lag)

  q = c(.025, .5, .975)
  ci <- pp_tidy |> 
    group_by(model) |> 
    summarize(pp_median = quantile(posterior, probs = q[2]),
              pp_lower = quantile(posterior, probs = q[1]), 
              pp_upper = quantile(posterior, probs = q[3]))  |> 
    mutate(lag = lag) |> 
    arrange(model)
  
  
  contrast_lists <- 
    if (dem_var == "sex") {
    c(list("male"), list("female"))
  } else if (dem_var == "income") {
    c(list("above poverty"), list("below poverty"))
  } else if (dem_var == "race") {
    c(list("non-hispanic white"), list("not white"))
  } else {
    stop(dem_var, " not in data")
  }
      
  ci_contrast <- pp |>
    contrast_models(contrast_lists[1],  contrast_lists[2]) |> 
  summary(size = 0) 
  
  ci_median_contrast <- pp |> 
    contrast_models(contrast_lists[1],  contrast_lists[2]) |>  
    group_by(contrast) |> 
    summarize(median = quantile(difference, .5)) |> 
    mutate(contrast = str_remove(contrast, "\\."))


ci_contrast <- ci_contrast |> 
    mutate(lag = lag) |> 
    left_join(ci_median_contrast, by = c("contrast")) |> 
    select(contrast, probability, median, lower, upper, lag) 
  
  list(pp = pp_tidy, ci = ci, ci_contrast = ci_contrast)
}

sex

In [ ]:
sex <- c(0, 24,72,168,336) |> 
  map(\(lag) calc_pp(lag, "sex")) 


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 6.5e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.65 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 4000 [  0%]  (Warmup)
Chain 1: Iteration:  400 / 4000 [ 10%]  (Warmup)
Chain 1: Iteration:  800 / 4000 [ 20%]  (Warmup)
Chain 1: Iteration: 1200 / 4000 [ 30%]  (Warmup)
Chain 1: Iteration: 1600 / 4000 [ 40%]  (Warmup)
Chain 1: Iteration: 2000 / 4000 [ 50%]  (Warmup)
Chain 1: Iteration: 2001 / 4000 [ 50%]  (Sampling)
Chain 1: Iteration: 2400 / 4000 [ 60%]  (Sampling)
Chain 1: Iteration: 2800 / 4000 [ 70%]  (Sampling)
Chain 1: Iteration: 3200 / 4000 [ 80%]  (Sampling)
Chain 1: Iteration: 3600 / 4000 [ 90%]  (Sampling)
Chain 1: Iteration: 4000 / 4000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.957 seconds (Warm-up)
Chain 1:                1.663 seconds (Sampling)
Chain 1:                3.62

# Posterior samples of performance
# A tibble: 80,000 × 3
   model  posterior   lag
   <chr>      <dbl> <dbl>
 1 male       0.936     0
 2 female     0.901     0
 3 male       0.940     0
 4 female     0.895     0
 5 male       0.933     0
 6 female     0.890     0
 7 male       0.936     0
 8 female     0.895     0
 9 male       0.939     0
10 female     0.869     0
# ℹ 79,990 more rows

# A tibble: 10 × 5
   model  pp_median pp_lower pp_upper   lag
   <chr>      <dbl>    <dbl>    <dbl> <dbl>
 1 female     0.889    0.864    0.911     0
 2 male       0.933    0.916    0.946     0
 3 female     0.866    0.837    0.891    24
 4 male       0.911    0.891    0.928    24
 5 female     0.853    0.822    0.880    72
 6 male       0.904    0.883    0.922    72
 7 female     0.827    0.791    0.858   168
 8 male       0.907    0.886    0.926   168
 9 female     0.797    0.753    0.835   336
10 male       0.895    0.867    0.916   336

# A tibble: 5 × 6
  contrast       probability median  lower  upper   lag
  <chr>                <dbl>  <dbl>  <dbl>  <dbl> <dbl>
1 male vs female           1 0.0429 0.0282 0.0591     0
2 male vs female           1 0.0443 0.0268 0.0629    24
3 male vs female           1 0.0511 0.0315 0.0718    72
4 male vs female           1 0.0800 0.0585 0.103    168
5 male vs female           1 0.0977 0.0732 0.125    336

income

In [ ]:
income <- c(0, 24,72,168,336) |> 
  map(\(lag) calc_pp(lag, "income")) 


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 2.4e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.24 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 4000 [  0%]  (Warmup)
Chain 1: Iteration:  400 / 4000 [ 10%]  (Warmup)
Chain 1: Iteration:  800 / 4000 [ 20%]  (Warmup)
Chain 1: Iteration: 1200 / 4000 [ 30%]  (Warmup)
Chain 1: Iteration: 1600 / 4000 [ 40%]  (Warmup)
Chain 1: Iteration: 2000 / 4000 [ 50%]  (Warmup)
Chain 1: Iteration: 2001 / 4000 [ 50%]  (Sampling)
Chain 1: Iteration: 2400 / 4000 [ 60%]  (Sampling)
Chain 1: Iteration: 2800 / 4000 [ 70%]  (Sampling)
Chain 1: Iteration: 3200 / 4000 [ 80%]  (Sampling)
Chain 1: Iteration: 3600 / 4000 [ 90%]  (Sampling)
Chain 1: Iteration: 4000 / 4000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.988 seconds (Warm-up)
Chain 1:                3.086 seconds (Sampling)
Chain 1:                5.07

# Posterior samples of performance
# A tibble: 80,000 × 3
   model         posterior   lag
   <chr>             <dbl> <dbl>
 1 above poverty     0.896     0
 2 below poverty     0.872     0
 3 above poverty     0.913     0
 4 below poverty     0.910     0
 5 above poverty     0.912     0
 6 below poverty     0.890     0
 7 above poverty     0.914     0
 8 below poverty     0.887     0
 9 above poverty     0.912     0
10 below poverty     0.903     0
# ℹ 79,990 more rows

# A tibble: 10 × 5
   model         pp_median pp_lower pp_upper   lag
   <chr>             <dbl>    <dbl>    <dbl> <dbl>
 1 above poverty     0.909    0.872    0.937     0
 2 below poverty     0.897    0.854    0.927     0
 3 above poverty     0.890    0.855    0.917    24
 4 below poverty     0.870    0.831    0.902    24
 5 above poverty     0.880    0.843    0.909    72
 6 below poverty     0.857    0.816    0.891    72
 7 above poverty     0.868    0.823    0.903   168
 8 below poverty     0.855    0.807    0.893   168
 9 above poverty     0.851    0.799    0.894   336
10 below poverty     0.811    0.750    0.863   336

# A tibble: 5 × 6
  contrast                       probability median     lower  upper   lag
  <chr>                                <dbl>  <dbl>     <dbl>  <dbl> <dbl>
1 above poverty vs below poverty       0.848 0.0124 -0.00709  0.0330     0
2 above poverty vs below poverty       0.955 0.0198  0.000549 0.0399    24
3 above poverty vs below poverty       0.960 0.0227  0.00160  0.0448    72
4 above poverty vs below poverty       0.807 0.0122 -0.0118   0.0370   168
5 above poverty vs below poverty       0.980 0.0391  0.00752  0.0732   336

race

Filter out one NA row prior to race calculation for 0 - 168 lag

In [ ]:
auroc_dem_0 <- auroc_dem_0 |> 
  filter(!is.na(`not white`))

auroc_dem_24 <- auroc_dem_24 |> 
  filter(!is.na(`not white`))

auroc_dem_72 <- auroc_dem_72 |> 
  filter(!is.na(`not white`))

auroc_dem_168 <- auroc_dem_168 |> 
  filter(!is.na(`not white`))

Filter out 3 NA row prior to race calculation for 336 lag

In [ ]:
auroc_dem_336 <- auroc_dem_336 |> 
  filter(!is.na(`not white`))

In [ ]:
race <- c(0, 24,72,168,336) |> 
  map(\(lag) calc_pp(lag, "race")) 


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 2.3e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 4000 [  0%]  (Warmup)
Chain 1: Iteration:  400 / 4000 [ 10%]  (Warmup)
Chain 1: Iteration:  800 / 4000 [ 20%]  (Warmup)
Chain 1: Iteration: 1200 / 4000 [ 30%]  (Warmup)
Chain 1: Iteration: 1600 / 4000 [ 40%]  (Warmup)
Chain 1: Iteration: 2000 / 4000 [ 50%]  (Warmup)
Chain 1: Iteration: 2001 / 4000 [ 50%]  (Sampling)
Chain 1: Iteration: 2400 / 4000 [ 60%]  (Sampling)
Chain 1: Iteration: 2800 / 4000 [ 70%]  (Sampling)
Chain 1: Iteration: 3200 / 4000 [ 80%]  (Sampling)
Chain 1: Iteration: 3600 / 4000 [ 90%]  (Sampling)
Chain 1: Iteration: 4000 / 4000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.811 seconds (Warm-up)
Chain 1:                2.011 seconds (Sampling)
Chain 1:                3.82

# Posterior samples of performance
# A tibble: 80,000 × 3
   model              posterior   lag
   <chr>                  <dbl> <dbl>
 1 non-hispanic white     0.938     0
 2 not white              0.830     0
 3 non-hispanic white     0.937     0
 4 not white              0.824     0
 5 non-hispanic white     0.917     0
 6 not white              0.853     0
 7 non-hispanic white     0.893     0
 8 not white              0.758     0
 9 non-hispanic white     0.922     0
10 not white              0.764     0
# ℹ 79,990 more rows

# A tibble: 10 × 5
   model              pp_median pp_lower pp_upper   lag
   <chr>                  <dbl>    <dbl>    <dbl> <dbl>
 1 non-hispanic white     0.916    0.845    0.955     0
 2 not white              0.783    0.650    0.875     0
 3 non-hispanic white     0.894    0.801    0.949    24
 4 not white              0.727    0.558    0.851    24
 5 non-hispanic white     0.886    0.820    0.928    72
 6 not white              0.727    0.611    0.818    72
 7 non-hispanic white     0.874    0.810    0.922   168
 8 not white              0.747    0.637    0.835   168
 9 non-hispanic white     0.850    0.783    0.901   336
10 not white              0.718    0.617    0.802   336

# A tibble: 5 × 6
  contrast                        probability median  lower upper   lag
  <chr>                                 <dbl>  <dbl>  <dbl> <dbl> <dbl>
1 non-hispanic white vs not white       0.998  0.131 0.0567 0.222     0
2 non-hispanic white vs not white       0.999  0.165 0.0716 0.280    24
3 non-hispanic white vs not white       1      0.157 0.0866 0.236    72
4 non-hispanic white vs not white       0.999  0.125 0.0571 0.204   168
5 non-hispanic white vs not white       0.998  0.130 0.0575 0.208   336

Bind all pp/contrast tibbles and save

In [ ]:
posteriors_sex |> 
  bind_rows(posteriors_income) |> 
  bind_rows(posteriors_race) |> 
  write_csv(here::here(path_models_lag, "posteriors_dem.csv"))

pp_sex |> 
  bind_rows(pp_income) |> 
  bind_rows(pp_race) |> 
  write_csv(here::here(path_models_lag, "pp_dem_all.csv"))

pp_dem_contrast <- contrast_sex |> 
  bind_rows(contrast_income) |> 
  bind_rows(contrast_race) |> 
  write_csv(here::here(path_models_lag, "pp_dem_contrast_all.csv"))